In [1]:
%pip install -U -q selenium webdriver-manager easygui tenacity tqdm pandas openpyxl

import os
from time import sleep
import time
import json
import re
from datetime import datetime

from tqdm import tqdm
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from easygui import fileopenbox, boolbox, choicebox
from tenacity import stop_after_attempt, retry

USERID = 'louhuynh'
PASSWORD = 'Rbg8080@@'
SITEPASS = 'louiev8080'

In [2]:
def is_element_present(driver, by, value):
    try:
        driver.find_element(by, value)
        return True
    except NoSuchElementException:
        return False

#Read Input File
use_input = boolbox(msg='Do you want to use an Input File?', choices=("[Y]es", "[N]o"), default_choice='[Y]es')

if use_input:
    #Read Input File
    input_file_path = None
    while not input_file_path:
        input_file_path = fileopenbox('Select Input File')
    input_df = pd.read_excel(input_file_path, dtype='object').rename(columns={'Policy Number': 'Group Number'})
    input_df = input_df.drop_duplicates(ignore_index=True)
    input_df = input_df[['Group Number', 'Group Name']].copy(deep=True)
    status_df = input_df.copy()

#Select Report Type
report_types = {
                'Benefit Detail Report': 'Benefit Detail Report.csv',
                'Benefit Summary Report': 'Benefit Summary Report.csv',
                'Group Information Report': 'Group Information Report.pdf',
                'Over Age Dependent Report': 'Benefit Oad Report.csv'
                }
report_type_selected = None
while not report_type_selected:
    report_type_selected = choicebox(title='Select Report Type', msg='Select Report Type', choices=list(report_types.keys()), preselect=7)


#Set Download Location
today = datetime.today().strftime('%m-%d-%Y')
download_path = os.path.join(os.path.realpath(os.path.abspath('')), f'Downloads_{today}_{report_type_selected}s')
if not os.path.exists(download_path):
    os.mkdir(download_path)

#Spawn driver    
def start_browser():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.page_load_strategy='eager'
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option("useAutomationExtension", False)
    prefs = {"download.default_directory" : download_path}
    chrome_options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    driver.execute_cdp_cmd(
        "Page.addScriptToEvaluateOnNewDocument",
        {
            "source": """
            try {
                Object.defineProperty(window, 'navigator', {
                    value: new Proxy(navigator, {
                        has: (target, key) => (key === 'webdriver' ? false : key in target),
                        get: (target, key) => {
                            if (key === 'webdriver') {
                                return undefined;
                            }
                            const value = target[key];
                            return typeof value === 'function' ? value.bind(target) : value;
                        }
                    })
                });
            } catch (e) {
                console.error('Error redefining navigator:', e);
            }
            """
        }
    )

    driver.maximize_window()
    return driver

driver = start_browser()

LOGIN

In [3]:
driver.get('https://agentpoint.bcbsfl.com/apss/ps?locale=en')
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'NameCallback')))
driver.find_element(By.ID, "NameCallback").send_keys(USERID)
driver.find_element(By.ID, "PasswordCallback").send_keys(PASSWORD + Keys.ENTER)

try:
    #Site Password Login
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//span[contains(text(), "Skip for now")]')))
    if is_element_present(driver, By.XPATH, '//span[contains(text(), "Skip for now")]'):
        driver.find_element(By.CSS_SELECTOR, '[type="button"]').click()
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//span[contains(text(), "Enter Site Password")]')))
    driver.find_element(By.ID, "PasswordCallback").send_keys(SITEPASS + Keys.ENTER)
except:
    #MFA Login
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//span[contains(text(), "Multi-Factor Authentication")]')))
    if is_element_present(driver, By.XPATH, '//span[contains(text(), "Multi-Factor Authentication")]'):
        humaninput = boolbox(msg=f"MFA Login Stage detected. Have you logged in?",
                title=" ", choices=("[Y]es", "[N]o"), default_choice='[Y]es', cancel_choice='[N]o')

#Check Login Status
try:
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, '[data-icon="user"]')))
except:
    if is_element_present(driver, By.CSS_SELECTOR, 'img[alt="Pega Logo"]'):
        driver.back()
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, '[data-icon="user"]')))

NAVIGATE TO GROUP SEARCH PAGE

In [4]:
driver.get('https://agentpoint.bcbsfl.com/apss/ps/Business')

WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, '[data-icon="user-tag"]')))
driver.find_element(By.CSS_SELECTOR, '[data-icon="user-tag"]').click()
driver.close()
driver.switch_to.window(driver.window_handles[-1])

WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input[placeholder="Search by Group Name/Number"]')))

#Parse All Groups
json_string = re.search(r'window.agentInfo\s*=\s*(.+);\s*window', str(driver.page_source)).group(1)
jsondata = json.loads(json_string)
parsed_input = pd.DataFrame(data=jsondata['groups'])
parsed_input = parsed_input[['groupNumber', 'businessName']].rename(columns={'groupNumber': 'Group Number', 'businessName': 'Group Name'})
writer = pd.ExcelWriter(f'All Groups_{today}.xlsx', engine='auto')
parsed_input.to_excel(writer, index=False)
writer.close()

#If no input provided, proceed with Parsed Groups
if not use_input:
    input_df = parsed_input[['Group Number', 'Group Name']].copy(deep=True)
    status_df = parsed_input.copy()

driver.find_element(By.CSS_SELECTOR, 'input[placeholder="Search by Group Name/Number"]').click()

Downloader Functions

In [5]:
def checkFileDownloaded(download_path, fname, group_num):
    timeout = 15
    start_time = time.time()
    file_path = os.path.join(download_path, fname)
    renamed_path = os.path.join(download_path, f'{group_num}_{report_types[report_type_selected]}')
    while not os.path.exists(file_path):
        if time.time() - start_time > timeout:
            raise Exception
        time.sleep(0.5)
    try:
        os.rename(file_path, renamed_path)
    except FileExistsError:
        os.remove(os.path.join(download_path, f'{group_num}_{report_types[report_type_selected]}'))
        os.rename(file_path, renamed_path)
    return renamed_path

def processDownloadedFile(file_path, group_number, group_name):
    df = pd.read_csv(file_path, skiprows=6)
    df['Group Number'] = group_number
    df['Group Name'] = group_name
    return df

def saveOutput(final_output, processed_data, fname):
    '''Processes OAD Files ONLY'''
    try:
        final_output = pd.read_excel(fname, dtype='object')
    except:
        pass

    processed_data.rename(columns=lambda x: x.strip(), inplace=True)
    final_output = pd.concat([final_output, processed_data], axis=0, join='outer', ignore_index=True)
    final_output = final_output.fillna('').drop_duplicates(ignore_index=True)
    final_output = final_output.astype(str).map(lambda x: x.strip(), na_action='ignore').drop_duplicates(ignore_index=True)

    writer = pd.ExcelWriter(fname, engine='auto')
    final_output.to_excel(writer, sheet_name='Data', index=False)
    dependent_children_df = final_output[final_output['Relationship']=='Dependent Child']   
    dependent_children_df.to_excel(writer, sheet_name='OAD', index=False)
    writer.close()

GROUP NUMBER SEARCH

In [20]:
final_output = pd.DataFrame(columns=['Group Number', 'Group Name'])

@retry(stop=stop_after_attempt(3))
def processGroup(_, group_number, group_name):
    group_number = re.sub(r'[^A-Z\d]','', str(group_number))
    try:
        close_buttons = driver.find_elements(By.CSS_SELECTOR, 'button.sgl-core-notification--close-button')
        for button in close_buttons:
            button.click()
    except:
        pass
    driver.find_element(By.CSS_SELECTOR, 'span[data-icon="list"]').click()
    driver.find_element(By.CSS_SELECTOR, 'input[placeholder="Search by Group Name/Number"]').send_keys(Keys.BACK_SPACE*20 + f'{group_number}')
    try:
        group_name = driver.find_element(By.CSS_SELECTOR, 'tbody tr.sgl-core-table--row td').text
    except:
        no_results = driver.find_element(By.CSS_SELECTOR, 'div.een-components-search-tool-window--no-search-results')
        if no_results:
            return pd.DataFrame()

    driver.find_element(By.CSS_SELECTOR, 'tbody tr.sgl-core-table--row').click()
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, '[data-icon="bar-chart"]')))

    # Handle Rate Verification
    try:
        driver.find_element(By.CSS_SELECTOR, '[data-icon="bar-chart"]').click()    
    except:
        driver.find_element(By.CSS_SELECTOR,'div.sgl-core-confirm--content span.sgl-core-button--text').click()
        driver.find_element(By.CSS_SELECTOR, '[data-icon="bar-chart"]').click()  

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'span.sgl-text--soft-black')))
    sleep(0.5)
    #Select Report to Click
    report_elems = [{elem.text: elem} for elem in driver.find_elements(By.CSS_SELECTOR, 'span.sgl-text--soft-black')]
    report_elems = {k: v for d in report_elems for k, v in d.items()}
    report_elems[report_type_selected].click()

    #Remove stray downloaded files
    pattern = re.compile(f'{report_types[report_type_selected].split(".")[0]}.*')
    for filename in os.listdir(download_path):
        if pattern.match(filename):
            os.remove(os.path.join(download_path, filename))

    #Click Generate Report
    driver.find_elements(By.CSS_SELECTOR, 'div.aba-sections-ba-reports--button-controls button')[-1].click()

    if report_type_selected == 'Benefit Detail Report':
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//span[contains(text(), "Select All")]')))
        driver.find_elements(By.CSS_SELECTOR, 'span.sgl-text--soft-black')[-1].click()
        driver.find_element(By.XPATH, '//span[contains(text(), "Select All")]').click()
        driver.find_elements(By.CSS_SELECTOR, 'div.aba-sections-ba-reports--button-controls button')[-1].click()

    empty_report_timeouts = {
                    'Benefit Detail Report': 2.5,
                    'Benefit Summary Report': 7.5,
                    'Group Information Report': 15,
                    'Over Age Dependent Report': 7.5
                    }

    try:
        # Check if file not downloadable
        empty_report_timeout = empty_report_timeouts[report_type_selected]
        WebDriverWait(driver, empty_report_timeout).until(EC.visibility_of_element_located((By.XPATH, '//p[contains(text(), "Empty Report")]')))
        WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.sgl-core-notification--close-button')))
        close_buttons = driver.find_elements(By.CSS_SELECTOR, 'button.sgl-core-notification--close-button')
        for button in close_buttons:
            button.click()
        if report_type_selected == 'Over Age Dependent Report':
            return pd.DataFrame()
        else:
            return False

    except:
        #Download and rename file to Group Number
        downloaded_file = checkFileDownloaded(download_path, report_types[report_type_selected], group_number)
        if report_type_selected == 'Over Age Dependent Report':
            processed_data = processDownloadedFile(downloaded_file, group_number, group_name)
            return processed_data
        return True


#Process Rows
for row in tqdm(input_df.itertuples(), total=input_df.shape[0]):
    try:
        processed_data = processGroup(*tuple(row))
    except:
        status_df.at[row[0], 'Status'] = 'Unknown Error'
    #Handle OAD Statuses
    if report_type_selected == 'Over Age Dependent Report':
        if not processed_data.empty:
            saveOutput(final_output, processed_data, fname=f'Output_{today}.xlsx')
            status_df.at[row[0], 'Status'] = 'Downloaded'
        else:
            status_df.at[row[0], 'Status'] = 'Not Available'
        status_df.to_csv('Input File Status.csv', index=False)
    
    #Handle Non-OAD statuses
    else:
        if processed_data:
            status_df.at[row[0], 'Status'] = 'Downloaded'
        else:
            status_df.at[row[0], 'Status'] = 'Not Available'
        status_df.to_csv('Input File Status.csv', index=False)

  8%|▊         | 51/634 [03:19<38:02,  3.91s/it]
